# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
import joblib
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
import json
import numpy as np
from azureml.automl.core.shared import constants
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'my_experiment'

experiment=Experiment(ws, experiment_name)

#Environment(name="Auto-ML")

In [3]:
cpu_cluster_name = "auto-ml"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster Found!.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

Cluster Found!.


In [4]:
dataset = Dataset.get_by_name(ws, name='Capstone_Data')
Capstone_Dataset = dataset.to_pandas_dataframe()

Capstone_Dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "task":'classification',
    "primary_metric":'accuracy',
    "training_data":dataset,
    "label_column_name":'Outcome',
    "n_cross_validations":5,
    "max_concurrent_iterations": 4,
    "featurization": 'auto'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    **automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
my_experiment,AutoML_56ac2cae-8af3-4a48-b087-94d371fb4fa5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
my_experiment,AutoML_56ac2cae-8af3-4a48-b087-94d371fb4fa5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
remote_run.get_metrics('accuracy')

{'accuracy': 0.7774042950513538}

In [9]:
print(remote_run.get_portal_url())

https://ml.azure.com/runs/AutoML_56ac2cae-8af3-4a48-b087-94d371fb4fa5?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-239402/workspaces/quick-starts-ws-239402&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [10]:
best_run, best_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [11]:
print(best_run)

Run(Experiment: my_experiment,
Id: AutoML_56ac2cae-8af3-4a48-b087-94d371fb4fa5_51,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l1', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('35', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=T

In [13]:
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l1', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('35', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=109.85411419875572, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l1', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('23', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True

In [14]:
#TODO: Save the best model
joblib.dump(value=best_model, filename='best-automl2.pkl')

['best-automl2.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
#register the model

model = Model.register(workspace=ws,model_name = "best-automl", model_path = './best-automl.pkl')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
print(model.name, model.id, model.version, sep='\t')

Registering model best-automl
best-automl	best-automl:2	2


In [21]:
#create an inference config and deploy the model as a web service

env = Environment.from_conda_specification('myenv', 'myenv.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True,tags = {'name':'Diabetes Prediction'},description='Diabetes prediction model')
service = model.deploy(workspace=ws,name="automl-deploy5",models=[model],inference_config=inference_config,deployment_config=aci_config)


service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-07-29 16:21:55+00:00 Creating Container Registry if not exists.
2023-07-29 16:21:55+00:00 Registering the environment.
2023-07-29 16:21:56+00:00 Use the existing image.
2023-07-29 16:21:56+00:00 Generating deployment configuration.
2023-07-29 16:21:57+00:00 Submitting deployment to compute.
2023-07-29 16:22:02+00:00 Checking the status of deployment automl-deploy5..
2023-07-29 16:23:56+00:00 Checking the status of inference endpoint automl-deploy5.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [22]:
service.update(enable_app_insights=True)

In [23]:
print(service.state)

Healthy


In [24]:
print(service.scoring_uri)

http://4a03a3d6-cba1-47c7-bba6-9ff498c547eb.westeurope.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [25]:
row = 0
x = Capstone_Dataset.iloc[row, :-1].to_dict()
y = Capstone_Dataset.iloc[row, -1]



In [26]:
print(x)

{'Pregnancies': 6.0, 'Glucose': 148.0, 'BloodPressure': 72.0, 'SkinThickness': 35.0, 'Insulin': 0.0, 'BMI': 33.6, 'DiabetesPedigreeFunction': 0.627, 'Age': 50.0}


In [27]:
print(y)

1


In [29]:
#input_data = json.dumps({
    #'data': x
#})

#output = service.run(input_data)
#print(f"Predicted: {output}\nExpected: {y}")
#print(f"Output: {y}")

# URL for the web service
scoring_uri = 'http://4a03a3d6-cba1-47c7-bba6-9ff498c547eb.westeurope.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'hEvDebXsIOz0OvGInmyn5VqIKwMtUeE8'

data = {
  "data": [
    {
        'Pregnancies': 6.0, 
        'Glucose': 148.0, 
        'BloodPressure': 72.0, 
        'SkinThickness': 35.0, 
        'Insulin': 0.0,
        'BMI': 33.6, 
        'DiabetesPedigreeFunction': 0.627, 
        'Age': 50.
    }             
 ]
}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)
#print(f"Predicted: {output}\nExpected: {y}")

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction ", resp.text)

prediction  [1]


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
#print the logs of the web service
print(service.get_logs())
#delete the service
service.delete()

/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-07-29T16:26:11,549190132+00:00 - rsyslog/run 
2023-07-29T16:26:11,556726132+00:00 - gunicorn/run 
2023-07-29T16:26:11,563476832+00:00 | gunicorn/run | 
2023-07-29T16:26:11,570572232+00:00 | gunicorn/run | ###############################################
2023-07-29T16:26:11,572384732+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-07-29T16:26:11,578896132+00:00 | gunicorn/run | ###############################################
bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required b